# Predicting MOOC grade improvement using a LSTM recurrent neural network

### Importing data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.preprocessing import sequence
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [2]:
db = pd.read_csv("OutputTable.csv")
db.sort(columns=['UserID', 'ProblemID', 'SubmissionNumber'], inplace=True)
db['NVideoAndForum'] = db['NVideoEvents'] + db['NForumEvents']
db = db[db['NVideoAndForum'] > 0]
db['improved'] = (db['GradeDiff'] > 0).astype(int)
db['dummy'] = (db['improved'] < 1 ).astype(int) 
db.fillna(value=0, inplace=True)
#db.replace(to_replace=db['ProblemID'].values, value=np.arange(0,len(db)))

/Users/Thomas/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [3]:
db.set_index('UserID', inplace=True)
for user in db.index.values:
    sub = db.loc[user]
    sub['ProblemID'] = sub['ProblemID'] + hash(user)

/Users/Thomas/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Thomas/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
db.reset_index(inplace=True)
db.set_index(['UserID', 'ProblemID', 'SubmissionNumber'], inplace=True)
db.head()

TimeStamp  \
UserID                           ProblemID            SubmissionNumber               
0009e7ff2e6bad5d80640eeb61cd6e0d -5532758789028288788 13                1397499746   
                                                      14                1397502429   
                                 -5532758789028288785 5                 1397596602   
                                                      9                 1397596892   
                                                      10                1397597189   

                                                                        TimeSinceLast  \
UserID                           ProblemID            SubmissionNumber                  
0009e7ff2e6bad5d80640eeb61cd6e0d -5532758789028288788 13                        529.0   
                                                      14                       2683.0   
                                 -5532758789028288785 5                         233.0   
                                                      9                          97.0   
                                                      10                        297.0   

                                                                            Grade  \
UserID                           ProblemID            SubmissionNumber              
0009e7ff2e6bad5d80640eeb61cd6e0d -5532758789028288788 13                88.888889   
                                                      14                77.777778   
                                 -5532758789028288785 5                 80.000000   
                                                      9                 86.666667   
                                                      10                86.666667   

                                                                        GradeDiff  \
UserID                           ProblemID            SubmissionNumber              
0009e7ff2e6bad5d80640eeb61cd6e0d -5532758789028288788 13                11.111111   
                                                      14               -11.111111   
                                 -5532758789028288785 5                  0.000000   
                                                      9                  0.000000   
                                                      10                 0.000000   

                                                                        NVideoEvents  \
UserID                           ProblemID            SubmissionNumber                 
0009e7ff2e6bad5d80640eeb61cd6e0d -5532758789028288788 13                         0.0   
                                                      14                         0.0   
                                 -5532758789028288785 5                          0.0   
                                                      9                          0.0   
                                                      10                         0.0   

                                                                        NForumEvents  \
UserID                           ProblemID            SubmissionNumber                 
0009e7ff2e6bad5d80640eeb61cd6e0d -5532758789028288788 13                         3.0   
                                                      14                        27.0   
                                 -5532758789028288785 5                          6.0   
                                                      9                          1.0   
                                                      10                         2.0   

                                                                        NumberOfPosts  \
UserID                           ProblemID            SubmissionNumber                  
0009e7ff2e6bad5d80640eeb61cd6e0d -5532758789028288788 13                          0.0   
                                                      14                          0.0   
                                 -5532758789028288785 5         

In [5]:
db.drop(['TimeStamp', 'Grade', 'GradeDiff'], axis=1, inplace=True)
db

TimeSinceLast  \
UserID                           ProblemID            SubmissionNumber                  
0009e7ff2e6bad5d80640eeb61cd6e0d -5532758789028288788 13                        529.0   
                                                      14                       2683.0   
                                 -5532758789028288785 5                         233.0   
                                                      9                          97.0   
                                                      10                        297.0   
                                                      11                      46568.0   
                                                      22                        320.0   
0020dea356e254c45d9d4ffe68229b3b  972832200708165442  1                         436.0   
                                  972832200708165448  1                       23045.0   
                                  972832200708165449  3                         499.0   
                                  972832200708165454  6                         416.0   
0123f0d5ea6362ba71ac18b22583d71d  1679523696307806018 2                        3840.0   
                                                      3                        2450.0   
                                                      4                       69276.0   
                                                      5                       10540.0   
                                                      7                        3425.0   
                                                      8                        1702.0   
                                                      9                         549.0   
                                                      10                        422.0   
                                  1679523696307806020 4                        6548.0   
                                                      7                         392.0   
                                                      9                         932.0   
                                                      10                        160.0   
                                                      11                        552.0   
                                                      15                      72049.0   
                                                      17                       3525.0   
                                                      20                       6182.0   
                                                      33                      80607.0   
                                                      34                     533164.0   
                                                      35                      57809.0   
...                                                                               ...   
fec803e81a96c3fdc65dfa6cc00e0cbb  4878710041789685395 3                       66877.0   
                                                      4                        1185.0   
                                                      12                       1404.0   
                                                      14                       1075.0   
                                                      15                     270160.0   
                                  4878710041789685397 3                        3540.0   
                                                      6                        1137.0   
                                                      7                      232456.0   
                                                      9                        2860.0   
                                                      13                       7159.0   
                                                      15                        691.0   
                                                      18                      72256.0   
                                                      19           

In [6]:
max_submission = db.index.levshape[2]
max_submission

99

In [11]:
a3d = np.array(list(db.groupby(level=1).apply(pd.DataFrame.as_matrix)))
X_train = sequence.pad_sequences(a3d, maxlen=max_submission, padding='post')
X_train.shape

(1907, 99, 19)

In [12]:
y_train = X_train[:,:,17:20]
X_train = X_train[:,:,:18]

In [14]:
X_train = X_train.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
for i in range(0,X_train.shape[0]):
    X_train[i,:,:] = scaler.fit_transform(X_train[i,:,:])

### LSTM network

(User*problem, submission, feature)

In [16]:
from keras.layers.core import Masking, TimeDistributedDense


In [19]:
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(99, 18)))
model.add(LSTM(50, return_sequences=True, activation='relu'))
#model.add(TimeDistributed(Dense(30, activation='sigmoid')))
model.add(TimeDistributedDense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/Users/Thomas/anaconda/envs/tensorflow/lib/python3.5/site-packages/keras/layers/core.py:1147: UserWarning: TimeDistributedDense is deprecated, please use TimeDistributed(Dense(...)) instead.
  warnings.warn('TimeDistributedDense is deprecated, '


In [20]:
model.fit(X_train,y_train,batch_size=100, nb_epoch=100)

Epoch 1/100
1907/1907 [==============================] - 13s - loss: 0.6668 - acc: 0.4827    
Epoch 2/100
1907/1907 [==============================] - 11s - loss: 0.6388 - acc: 0.5041    
Epoch 3/100
1907/1907 [==============================] - 11s - loss: 0.6130 - acc: 0.5062    
Epoch 4/100
1907/1907 [==============================] - 11s - loss: 0.5787 - acc: 0.5247    
Epoch 5/100
1907/1907 [==============================] - 11s - loss: 0.5375 - acc: 0.4275    
Epoch 6/100
1907/1907 [==============================] - 11s - loss: 0.4899 - acc: 0.4992    
Epoch 7/100
1907/1907 [==============================] - 11s - loss: 0.4277 - acc: 0.5031    
Epoch 8/100
1907/1907 [==============================] - 11s - loss: 0.3661 - acc: 0.5127    
Epoch 9/100
1907/1907 [==============================] - 11s - loss: 0.3025 - acc: 0.5120    
Epoch 10/100
1907/1907 [==============================] - 10s - loss: 0.2417 - acc: 0.5125    
Epoch 11/100
1907/1907 [==============================] - 1